# **Семинар 2 — Управление облачными ресурсами с помощью Terraform (IaC) и введение в Feature Store Feast**

## **Цель занятия**

Освоить базовые принципы **Infrastructure as Code (IaC)** и научиться разворачивать инфраструктуру ML-проекта с помощью **Terraform**, а также подключить к пайплайну **Feature Store (Feast)** для управления признаками и согласованности данных между обучением и продакшеном.

## **Задачи семинара**

- Разобраться с концепцией Infrastructure as Code (IaC).  
- Создать базовый Terraform-проект для ML-инфраструктуры.  
- Научиться разворачивать ресурсы AWS (EC2, S3).  
- Настроить хранение состояния Terraform в S3.  
- Ознакомиться с концепцией Feature Store и интеграцией Feast.  

## **План**
1. Краткое введение в IaC и Terraform  
2. Развёртывание базовой инфраструктуры  
3. Работа с backend и модулями  
4. Введение в Feature Store Feast  
5. Практическая работа и интеграция с Airflow


## **1. Введение в Infrastructure as Code (IaC)**

**Infrastructure as Code (IaC)** — подход, при котором инфраструктура описывается с помощью кода (файлов конфигурации), а не вручную через интерфейсы облачных провайдеров.

Преимущества IaC:
- Реплицируемость — одно и то же окружение можно развернуть где угодно.  
- Контроль версий — инфраструктура версионируется в Git.  
- Автоматизация — минимизация человеческого фактора.  
- Интеграция с CI/CD — инфраструктура разворачивается при пуше кода.

Наиболее популярные инструменты:
- **Terraform** (HashiCorp)
- **Pulumi**
- **AWS CloudFormation**
- **Ansible (в части конфигурации)**

На семинаре мы используем Terraform как основной инструмент IaC для ML-инфраструктуры.


## **2. Установка и подготовка Terraform-проекта**

Terraform состоит из:
- **provider** — отвечает за взаимодействие с облаком (AWS, GCP, Azure и др.);
- **resource** — создаваемые объекты (инстансы, хранилища, базы данных);
- **state** — текущее состояние инфраструктуры;
- **plan/apply** — команды для развертывания и обновления.

Установите Terraform (если локально):

```bash
# macOS
brew install terraform

# Ubuntu/Debian
sudo apt update && sudo apt install -y terraform

infra/
├── main.tf
├── variables.tf
├── outputs.tf


In [ ]:
# --- infra/main.tf ---
provider "aws" {
  region = "us-east-1"
}

resource "aws_instance" "ml_server" {
  ami           = "ami-0abcdef1234567890"
  instance_type = "t3.medium"
  tags = {
    Name = "ML-Server"
  }
}

resource "aws_s3_bucket" "ml_data" {
  bucket = "mlops-data-storage"
  acl    = "private"
}

output "server_ip" {
  value = aws_instance.ml_server.public_ip
}


## **3. Запуск и проверка Terraform**

Terraform работает по шагам:

1. **Инициализация** проекта:
```bash
terraform init

terraform plan

terraform apply


После подтверждения (нажмите yes) Terraform создаст:
EC2-инстанс с именем ML-Server;
S3-бакет для хранения данных.
Проверьте, что после выполнения команда terraform output возвращает IP созданного сервера.

## **4. Хранение состояния и использование backend**

По умолчанию Terraform хранит состояние (`terraform.tfstate`) локально, что неудобно для командной работы.

Чтобы состояние не потерялось и могло использоваться несколькими разработчиками, перенесём его в **S3**.

In [ ]:
# --- backend configuration ---
terraform {
  backend "s3" {
    bucket = "mlops-terraform-state"
    key    = "prod/terraform.tfstate"
    region = "us-east-1"
  }
}


После этого при каждом `terraform apply` состояние будет храниться централизованно в S3-бакете.

Преимущества удалённого состояния:
- команда работает с одной "истиной" (single source of truth);
- можно отслеживать изменения между версиями;
- легко интегрировать с CI/CD.

💡 **Совет:**  
Не храните `terraform.tfstate` в Git! Это чувствительный файл с секретами доступа к облаку.


## **5. Модульная структура Terraform**

Чтобы не дублировать код, Terraform поддерживает **модули**.

Пример структуры:
```bash
modules/
├── compute/
│ └── main.tf
└── storage/
└── main.tf


Модули можно переиспользовать в разных проектах, просто указав путь:
```hcl
module "compute" {
  source = "./modules/compute"
  instance_type = "t3.medium"
}


## **6. Введение в Feature Store (Feast)**

**Feature Store** — это компонент MLOps-платформы, который отвечает за хранение и управление признаками (features), используемыми при обучении и инференсе моделей.

Он решает проблему *несогласованности данных* между тренировкой и продакшеном.

### **Основные функции:**
- централизованное хранение фичей;
- контроль версий признаков;
- быстрый доступ к оффлайн и онлайн-данным;
- интеграция с ML-пайплайнами и оркестраторами.

Наиболее популярное open-source решение — **Feast (Feature Store by Tecton)**.


## **7. Конфигурация Feast**

Файл конфигурации `feature_store.yaml`:

```yaml
project: mlops_project
registry: data/registry.db
provider: local
online_store:
  type: sqlite
offline_store:
  type: file
  path: data/offline


Feast использует registry.db — SQLite-хранилище с метаданными признаков.
Для работы создайте директорию data/ и добавьте оффлайн-данные (например, CSV).

In [ ]:
# Пример интеграции Feast с pandas

from feast import FeatureStore
import pandas as pd

store = FeatureStore(repo_path=".")

# Пример DataFrame с новыми данными
data = pd.DataFrame({
    "customer_id": [101, 102],
    "age": [34, 28],
    "balance": [1200.5, 800.0],
})

# Получение фичей
features = store.get_online_features(
    features=["customer_features:age", "customer_features:balance"],
    entity_rows=[{"customer_id": 101}]
).to_dict()

print(features)


## **8. Интеграция Airflow и Feature Store**

Теперь, когда у нас есть инфраструктура и Feature Store, их можно объединить:
- Airflow DAG запускает обучение и переобучение модели;
- Terraform обеспечивает инфраструктуру (S3, EC2, DB);
- Feast хранит и предоставляет фичи.

Пример задачи в DAG:
```python
from feast import FeatureStore
store = FeatureStore(repo_path="/opt/feature_store")
features = store.get_historical_features(
    entity_df="SELECT * FROM customer_entities",
    features=["customer_features:balance", "customer_features:activity"]
).to_df()


## **9. Интерактивная часть**

**Практические шаги:**

1. Разверните Terraform-инфраструктуру (EC2 + S3).  
2. Настройте хранение состояния Terraform в S3.  
3. Установите и настройте локальный Feast.  
4. Добавьте признаки и извлеките их с помощью API.  
5. Интегрируйте обращение к Feature Store в свой Airflow DAG.  

**Вопрос для размышления:**  
Почему важно хранить признаки централизованно, а не пересчитывать при каждом обучении?


## **10. Итоги семинара**

После занятия вы научились:

- Использовать Terraform для автоматизации инфраструктуры ML-проектов.  
- Настраивать хранение состояния и модули.  
- Понимать принципы работы Feature Store и конфигурацию Feast.  
- Интегрировать инфраструктуру и данные в единый MLOps-конвейер.  

Следующий шаг — добавить мониторинг качества данных и моделей в Airflow-пайплайн и реализовать обратную связь с системой алертинга.
